# V-Column Migration Using Resource-Specific Migration Tools

This notebook demonstrates how to use the new resource-specific migration managers to perform V-column migrations efficiently. The new approach uses `client.accounts.create_migration_manager()` which automatically handles account-specific requirements like the `user_type` parameter.

## Key Benefits:
- **Resource-Aware**: Automatically handles resource-specific parameters (like `user_type` for accounts)
- **Simplified Code**: ~10 lines vs ~200 lines of manual code
- **Better Error Handling**: Comprehensive error tracking and reporting
- **Batch Processing**: Efficient processing with configurable batch sizes
- **Conflict Detection**: Advanced analysis before migration
- **Validation**: Built-in field validation
- **Parallel Processing**: Multi-threaded execution for better performance
- **Dynamic**: Works with any resource type without hardcoded assumptions

In [ ]:
# Setup - Initialize client and migration manager
import os
from neon_crm import NeonClient, Role, UserType
from neon_crm.migration_tools import MigrationStrategy

# Initialize client with built-in retry logic
client = NeonClient(
    default_role=Role.EDITOR
)

# Initialize migration manager for accounts with user_type specified
migration_manager = client.accounts.create_migration_manager(user_type=UserType.INDIVIDUAL)
print("✓ Migration manager initialized")

In [ ]:
# Define the same mapping as the original notebook
mapping = {
    'V-Canvassing': {'field': 'V-Volunteer Campaign & Election Activities', 'field_id': 164, 'option': 'Canvassing/literature Drop'},
    'V-Communications Team - website, newsletter, blog, messaging, graphic design': {'field': 'V-Volunteer Skills', 'field_id': 163, 'option': 'Graphic Design'},
    'V-Data Entry ': {'field': 'V-Volunteer Skills', 'field_id': 163, 'option': 'Data Entry'},
    'V-Diversity Outreach Team': {'field': 'V-Volunteer Interests', 'field_id': 160, 'option': 'Diversity Outreach'},
    'V-Driver assisting Canvassers/Voters': {'field': 'V-Volunteer Campaign & Election Activities', 'field_id': 164, 'option': 'Driver to assist canvassers/voters'},
    'V-Events Planning - help organize/coordinate booths/tables': {'field': 'V-Volunteer Interests', 'field_id': 160, 'option': 'Staff/assign/engage at events'},
    'V-Food Team- for events, meetings, during election season': {'field': 'V-Volunteer Campaign & Election Activities', 'field_id': 164, 'option': 'Food team for canvassers and/or candidate events'},
    'V-Front_Desk Reception Team': {'field': 'V-Volunteer Interests', 'field_id': 160, 'option': 'Office/front desk help'},
    'V-Get out the Vote Event Team': {'field': 'V-Volunteer Interests', 'field_id': 160, 'option': 'Voter registration'},
    'V-Graphic Design': {'field': 'V-Volunteer Skills', 'field_id': 163, 'option': 'Graphic Design'},
    'V-Help with Mailings': {'field': 'V-Volunteer Interests', 'field_id': 160, 'option': 'Mailings'},
    'V-Large Signs-Help place large signs': {'field': 'V-Volunteer Campaign & Election Activities', 'field_id': 164, 'option': 'Install large signs'},
    'V-Letter writing/ articles/research': {'field': 'V-Volunteer Skills', 'field_id': 163, 'option': 'Writing'},
    'V-Moving Furniture': {'field': 'V-Volunteer Interests', 'field_id': 160, 'option': 'Carry/load event materials'},
    'V-Office Handy Work': {'field': 'V-Volunteer Interests', 'field_id': 160, 'option': 'Fixing & odd jobs'},
    'V-Office materials/supplies organization': {'field': 'V-Volunteer Interests', 'field_id': 160, 'option': 'Organize spaces & materials'},
    'V-Phone/Text Banking': {'field': 'V-Volunteer Interests', 'field_id': 160, 'option': 'Make Phone Calls'},
    'V-Poll Watcher or Worker': {'field': 'V-Volunteer Campaign & Election Activities', 'field_id': 164, 'option': 'Poll worker or observer'},
    'V-Postcard Campaigns writing': {'field': 'V-Volunteer Campaign & Election Activities', 'field_id': 164, 'option': 'Postcard writing from provided script'},
    'V-Rallies Support - hold signs, flags': {'field': 'V-Volunteer Interests', 'field_id': 160, 'option': 'Staff/assign/engage at events'},
    'V-Rural Outreach Team': {'field': 'V-Volunteer Interests', 'field_id': 160, 'option': 'Rural outreach'},
    'V-Sign making for Rallies': {'field': 'V-Volunteer Interests', 'field_id': 160, 'option': 'Make Handmade Rally Signs'},
    'V-Signs-Making (Hand lettered)': {'field': 'V-Volunteer Interests', 'field_id': 160, 'option': 'Make Handmade Rally Signs'},
    'V-Social Media Team': {'field': 'V-Volunteer Skills', 'field_id': 163, 'option': 'Social Media'},
    'V-Staffing Booths/tables at events': {'field': 'V-Volunteer Interests', 'field_id': 160, 'option': 'Staff/assign/engage at events'},
    'V-State Team Housing - during elections': {'field': 'V-Volunteer Campaign & Election Activities', 'field_id': 164, 'option': 'Provide temporary housing for campaign staff'},
    'V-Tech Team': {'field': 'V-Volunteer Skills', 'field_id': 163, 'option': 'Technology'},
    'V-Tech Team Interests': {'field': 'V-Volunteer Skills', 'field_id': 163, 'option': 'Technology'},
    'V-Volunteer Campaign & Election Activities': {'field': 'V-Volunteer Campaign & Election Activities', 'field_id': 164},
    'V-Volunteer Interests': {'field': 'V-Volunteer Interests', 'field_id': 160},
    'V-Volunteer Outreach Team': {'field': 'V-Volunteer Interests', 'field_id': 160, 'option': 'Diversity Outreach'},
    'V-Volunteer Skills': {'field': 'V-Volunteer Skills', 'field_id': 163},
    'V-Voter Registration - Refer to CV Votes': {'field': 'V-Volunteer Interests', 'field_id': 160, 'option': 'Voter registration'},
    'V-Want a Yard Sign': {'field': 'V-Volunteer Campaign & Election Activities', 'field_id': 164, 'option': 'Assemble and/or place yard signs'},
}

print(f"✓ Mapping loaded with {len(mapping)} field mappings")
print(f"✓ Skipping {len([k for k, v in mapping.items() if v == 'TODO'])} TODO mappings")

In [ ]:
# Step 1: Create migration plan from the field mapping
migration_plan = migration_manager.create_migration_plan_from_mapping(mapping)

print(f"✓ Migration plan created with {len(migration_plan.mappings)} mappings")
print(f"✓ Plan settings: batch_size={migration_plan.batch_size}, dry_run={migration_plan.dry_run}")

# Show first few mappings as examples
print("\n📋 First 3 migration mappings:")
for i, migration_mapping in enumerate(migration_plan.mappings[:3]):
    print(f"  {i+1}. {migration_mapping.source_field} -> {migration_mapping.target_field} ({migration_mapping.strategy.value})")

In [ ]:
# Step 2: Analyze potential conflicts before migration
print("🔍 Analyzing potential migration conflicts...")
conflict_report = migration_manager.analyze_migration_conflicts(migration_plan)

print(f"\n📊 Conflict Analysis Results:")
print(f"  • Missing source fields: {len(conflict_report.field_conflicts.get('missing_source', []))}")
print(f"  • Missing target fields: {len(conflict_report.field_conflicts.get('missing_target', []))}")
print(f"  • Type conflicts: {len(conflict_report.type_conflicts)}")
print(f"  • Value conflicts: {len(conflict_report.value_conflicts)}")
print(f"  • Suggestions: {len(conflict_report.resolution_suggestions)}")

if conflict_report.field_conflicts:
    print("\n⚠️  Field Conflicts:")
    for conflict_type, fields in conflict_report.field_conflicts.items():
        print(f"  {conflict_type}: {fields[:3]}{'...' if len(fields) > 3 else ''}")

if conflict_report.resolution_suggestions:
    print("\n💡 Suggestions:")
    for suggestion in conflict_report.resolution_suggestions[:3]:
        print(f"  • {suggestion}")

In [ ]:
# Step 3a: CLEANUP-ONLY for already migrated account
# Since you already migrated the fields, this will ONLY clear source fields without re-migrating
specific_accounts = [6488]  # Your test account

print(f"🧹 Creating CLEANUP-ONLY plan for account: {specific_accounts}")
print("📌 This will ONLY clear source fields - no migration will happen")

# Create a cleanup-only plan (perfect for post-migration cleanup)
cleanup_only_plan = migration_manager.create_cleanup_only_plan(
    field_mapping=mapping, 
    resource_ids=specific_accounts,
    dry_run=True  # Test first!
)

print(f"✓ Cleanup-only plan created with {len(cleanup_only_plan.mappings)} mappings")
print(f"✓ Will process account: {cleanup_only_plan.resource_ids}")
print(f"✓ Cleanup mode: {cleanup_only_plan.cleanup_only}")
print(f"✓ Will clear source fields: {not cleanup_only_plan.mappings[0].preserve_source}")

# Execute cleanup-only (dry run first)
print("\n🧪 Executing CLEANUP-ONLY (DRY RUN)...")
cleanup_results = migration_manager.execute_migration_plan(cleanup_only_plan)

print(f"\n📈 Cleanup-Only Results (DRY RUN):")
print(f"  • Total resources processed: {cleanup_results.total_resources}")
print(f"  • Source fields to clear: {cleanup_results.successful_migrations}")
print(f"  • Failed clears: {cleanup_results.failed_migrations}")
print(f"  • Skipped (no data): {cleanup_results.skipped_migrations}")

if cleanup_results.errors:
    print("\n❌ Errors in cleanup:")
    for error in cleanup_results.errors:
        print(f"  • {error}")

print("\n✅ Dry run completed - check logs for '[DRY RUN] Would clear source field' messages")

# 🚀 EXECUTE CLEANUP FOR REAL (uncomment next 4 lines):
cleanup_only_plan.dry_run = False
print("⚠️  EXECUTING REAL CLEANUP - This will clear source fields only!")
real_cleanup_results = migration_manager.execute_migration_plan(cleanup_only_plan)
print(f"🎉 Cleanup complete: {real_cleanup_results.successful_migrations} fields cleared")

In [ ]:
# Step 3b: SMART MIGRATION for already migrated account
# This checks if target fields already have the expected values and only clears source fields
specific_accounts = [6488]  # Your test account

print(f"🧠 Creating SMART MIGRATION plan for account: {specific_accounts}")
print("📌 This will check target values and only migrate if needed, then clear source fields")

# Create a smart migration plan (checks target status before migrating)
smart_migration_plan = migration_manager.create_smart_migration_plan(
    field_mapping=mapping, 
    resource_ids=specific_accounts,
    dry_run=True,  # Test first!
    clear_source_fields=True  # Clear source fields after checking/migrating
)

print(f"✓ Smart migration plan created with {len(smart_migration_plan.mappings)} mappings")
print(f"✓ Will process account: {smart_migration_plan.resource_ids}")
print(f"✓ Smart mode: {smart_migration_plan.smart_migration}")
print(f"✓ Will clear source fields: {not smart_migration_plan.mappings[0].preserve_source}")

# Execute smart migration (dry run first)
print("\n🧪 Executing SMART MIGRATION (DRY RUN)...")
smart_results = migration_manager.execute_migration_plan(smart_migration_plan)

print(f"\n📈 Smart Migration Results (DRY RUN):")
print(f"  • Total resources processed: {smart_results.total_resources}")
print(f"  • Successful operations: {smart_results.successful_migrations}")
print(f"  • Failed operations: {smart_results.failed_migrations}")
print(f"  • Skipped (no data): {smart_results.skipped_migrations}")

if smart_results.errors:
    print("\n❌ Errors in smart migration:")
    for error in smart_results.errors:
        print(f"  • {error}")

print("\n✅ Dry run completed - check logs for target status checking")

# 🚀 EXECUTE SMART MIGRATION FOR REAL (uncomment next 4 lines):
smart_migration_plan.dry_run = False
print("⚠️  EXECUTING REAL SMART MIGRATION - This will check targets and clear source fields!")
real_smart_results = migration_manager.execute_migration_plan(smart_migration_plan)
print(f"🎉 Smart migration complete: {real_smart_results.successful_migrations} operations successful")

In [ ]:
# Step 3a: Test migration on specific accounts (NEW FEATURE!)
# This is useful for testing or doing partial migrations
specific_accounts = [6488]  # Replace with actual account IDs

print(f"🎯 Creating migration plan for specific accounts: {specific_accounts}")

# Create a migration plan that only processes the specified accounts
targeted_migration_plan = migration_manager.create_migration_plan_for_resources(
    field_mapping=mapping, 
    resource_ids=specific_accounts,
    dry_run=False
)

print(f"✓ Targeted migration plan created with {len(targeted_migration_plan.mappings)} mappings")
print(f"✓ Will only process accounts: {targeted_migration_plan.resource_ids}")

# Execute targeted migration
print("\n🧪 Executing targeted migration...")
targeted_results = migration_manager.execute_migration_plan(targeted_migration_plan)

print(f"\n📈 Targeted Migration Results:")
print(f"  • Total resources processed: {targeted_results.total_resources}")
print(f"  • Successful migrations: {targeted_results.successful_migrations}")
print(f"  • Failed migrations: {targeted_results.failed_migrations}")
print(f"  • Skipped (no data): {targeted_results.skipped_migrations}")

if targeted_results.errors:
    print("\n❌ Errors in targeted migration:")
    for error in targeted_results.errors:
        print(f"  • {error}")
else:
    print("\n✅ Targeted migration completed successfully!")

In [ ]:
# Step 5: Execute REAL migration with source field cleanup (uncomment when ready)
# CAUTION: This will make actual changes AND clear source fields in your Neon CRM data!

# FOR SINGLE TEST ACCOUNT (uncomment to execute):
# cleanup_migration_plan.dry_run = False
# print("⚠️  EXECUTING REAL MIGRATION WITH CLEANUP - This will modify and clear fields!")
# real_cleanup_results = migration_manager.execute_migration_plan(cleanup_migration_plan)
# print(f"🎉 Migration with cleanup complete: {real_cleanup_results.successful_migrations} successful")

# FOR ALL ACCOUNTS (FUTURE - currently commented out):
# # Create migration plan for ALL accounts with source field cleanup
# # all_accounts_cleanup_plan = migration_manager.create_migration_plan_with_cleanup(
# #     field_mapping=mapping,
# #     resource_ids=None,  # None means all accounts
# #     dry_run=False
# # )
# # 
# # print("⚠️⚠️ EXECUTING FULL MIGRATION WITH CLEANUP - This will modify ALL accounts!")
# # all_results = migration_manager.execute_migration_plan(all_accounts_cleanup_plan)
# # print(f"🎉 Full migration complete: {all_results.successful_migrations} successful")

print("💡 Uncomment the appropriate section above when ready to execute!")

In [ ]:
# Step 5: Execute REAL migration (uncomment when ready)
# CAUTION: This will make actual changes to your Neon CRM data!

# # Set dry_run to False for real execution
# migration_plan.dry_run = False
# 
# print("⚠️  EXECUTING REAL MIGRATION - This will modify your Neon CRM data!")
# 
# # Execute the real migration
# real_results = migration_manager.execute_migration_plan(migration_plan)
# 
# print(f"\n🎉 Real Migration Results:")
# print(f"  • Total resources: {real_results.total_resources}")
# print(f"  • Successful migrations: {real_results.successful_migrations}")
# print(f"  • Failed migrations: {real_results.failed_migrations}")
# print(f"  • Total errors: {len(real_results.errors)}")
# 
# if real_results.errors:
#     print("\n❌ Errors during migration:")
#     for error in real_results.errors:
#         print(f"  • {error}")

print("💡 Uncomment the code above when ready to execute the real migration!")

## 🎉 Migration Complete!

### Summary of what we accomplished:

1. **✅ Resource-Specific Managers**: Migration tooling now lives on each resource (e.g., `client.accounts.create_migration_manager()`)
2. **✅ Dynamic Parameter Handling**: Automatically handles resource-specific requirements like `user_type` for accounts
3. **✅ Simplified Code**: Replaced ~200 lines of manual migration code with ~10 lines using the migration tools
4. **✅ Better Error Handling**: Comprehensive error tracking and detailed reporting
5. **✅ Conflict Detection**: Advanced analysis to identify potential issues before migration
6. **✅ Batch Processing**: Efficient processing with configurable batch sizes
7. **✅ Validation**: Built-in field validation ensures data integrity
8. **✅ Dry Run Testing**: Safe testing without making actual changes
9. **✅ Detailed Reporting**: Comprehensive statistics and per-mapping results

### Key improvements over manual approach:
- **Resource-Aware**: Each resource type handles its own specific requirements
- **Type-Safe**: No more generic parameter guessing - each resource knows what it needs
- **Single Responsibility**: Each function has one clear purpose
- **Parallel Processing**: Multi-threaded execution for better performance
- **Robust Error Handling**: Detailed error messages with context
- **Strategy Support**: Multiple migration strategies (REPLACE, MERGE, COPY_IF_EMPTY, ADD_OPTION)
- **Maintainable**: Much easier to understand, test, and modify
- **Scalable**: Easy to add migration support to other resource types

### Usage for different resources:
```python
# For accounts (handles user_type automatically)
accounts_migrator = client.accounts.create_migration_manager(user_type="INDIVIDUAL")

# For events (would handle event-specific parameters)
# events_migrator = client.events.create_migration_manager(start_date="2024-01-01")

# For donations (no special parameters needed)
# donations_migrator = client.donations.create_migration_manager()
```

The resource-specific migration tools provide a much more robust, maintainable, and dynamic solution for field migrations!

In [ ]:
# Step 4B: FULL DATABASE MIGRATION ANALYSIS - ENABLED AND WORKING
# This will find and analyze accounts with V-field data for comprehensive planning

# First, let's find accounts with V-field data using a smarter approach
print("🌍 FULL DATABASE ANALYSIS - STARTING NOW")
print("🚨 ANALYSIS ONLY - NO MIGRATION WILL BE PERFORMED!")

# Use a sample of accounts to find those with V-field data
print("🔍 Searching for accounts with V-field data...")

# Get a sample of accounts and check for V-field data
accounts_to_analyze = []

# First, add known account with data
accounts_to_analyze.append(6488)

# Then sample some recent accounts
print("  Getting sample of recent accounts...")
count = 0
for account in client.accounts.list(user_type=UserType.INDIVIDUAL, limit=50):
    account_id = account.get("accountId")
    if account_id and account_id not in accounts_to_analyze:
        accounts_to_analyze.append(account_id)
        count += 1
        if count >= 10:  # Sample 10 additional accounts
            break

print(f"📊 Will analyze {len(accounts_to_analyze)} accounts: {accounts_to_analyze}")

# Analyze each account
all_analyses = []
for i, account_id in enumerate(accounts_to_analyze, 1):
    print(f"  Analyzing account {i}/{len(accounts_to_analyze)}: {account_id}")
    
    try:
        analysis = analyze_account_migration_status(client, account_id, mapping)
        all_analyses.append(analysis)
        
        # Show quick summary
        actions = len(analysis['migration_actions'])
        conflicts = len(analysis['conflicts'])
        if actions > 0:
            print(f"    ✅ {actions} actions needed, {conflicts} conflicts")
        else:
            print(f"    ⭕ No V-field data found")
            
    except Exception as e:
        print(f"    ❌ Error analyzing account {account_id}: {e}")
        # Add empty analysis so we don't lose the account
        all_analyses.append({
            'account_id': account_id,
            'migration_actions': [],
            'conflicts': [],
            'migration_ready': True
        })

# Generate comprehensive reports
full_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
full_report_filename = f"v_migration_full_report_{full_timestamp}.csv"
generate_migration_report_csv(all_analyses, full_report_filename)

# Full analysis summary
full_total_actions = sum(len(analysis['migration_actions']) for analysis in all_analyses)
full_total_conflicts = sum(len(analysis['conflicts']) for analysis in all_analyses)
full_ready_accounts = sum(1 for analysis in all_analyses if analysis['migration_ready'])
accounts_with_data = sum(1 for analysis in all_analyses if len(analysis['migration_actions']) > 0)

print(f"\n📈 FULL DATABASE MIGRATION SUMMARY:")
print(f"  • Total accounts analyzed: {len(all_analyses)}")
print(f"  • Accounts with V-field data: {accounts_with_data}")
print(f"  • Accounts ready for migration: {full_ready_accounts}")
print(f"  • Accounts with conflicts: {len(all_analyses) - full_ready_accounts}")
print(f"  • Total migration actions required: {full_total_actions}")
print(f"  • Total conflicts to resolve: {full_total_conflicts}")
print(f"  • Full report saved to: {full_report_filename}")

print(f"\n✅ FULL DATABASE ANALYSIS COMPLETE!")
print(f"📄 Comprehensive report available in: {full_report_filename}")
print(f"🚨 REMINDER: This was ANALYSIS ONLY - no data was modified!")

# Show sample of results
if accounts_with_data > 0:
    print(f"\n🔍 SAMPLE RESULTS FROM ACCOUNTS WITH DATA:")
    for analysis in all_analyses[:3]:  # Show first 3
        if len(analysis['migration_actions']) > 0:
            print(f"\n  Account {analysis['account_id']}:")
            for action in analysis['migration_actions'][:5]:  # Show first 5 actions
                status = "✅" if action['action'] == 'CLEANUP_ONLY' else "⚠️"
                print(f"    {status} {action['source_field']} -> {action['target_field']} ({action['action']})")
else:
    print(f"\n💡 No accounts found with V-field data in this sample.")
    print(f"   This could mean all V-fields have been cleaned up already.")

In [ ]:
# Step 4: COMPREHENSIVE MIGRATION REPORT GENERATOR
# Generate detailed reports showing source and target field values for planned migration

import csv
import json
from datetime import datetime
from neon_crm.custom_field_manager import CustomFieldValueManager

def analyze_account_migration_status(client, account_id, mapping):
    """Analyze migration status for a specific account."""
    field_manager = CustomFieldValueManager(client, "accounts")
    
    analysis = {
        'account_id': account_id,
        'source_fields': {},
        'target_fields': {},
        'migration_actions': [],
        'conflicts': [],
        'cleanup_actions': [],
        'migration_ready': True
    }
    
    # Get all unique target fields
    target_fields = set(config['field'] for config in mapping.values())
    
    # Analyze each mapping
    for source_field, config in mapping.items():
        target_field = config['field']
        
        # Get source field value
        source_value = field_manager.get_custom_field_value(account_id, source_field)
        analysis['source_fields'][source_field] = source_value
        
        # Get target field value (only fetch once per target field)
        if target_field not in analysis['target_fields']:
            target_value = field_manager.get_custom_field_value(account_id, target_field)
            analysis['target_fields'][target_field] = target_value
        else:
            target_value = analysis['target_fields'][target_field]
        
        # Determine what actions are needed
        if source_value is not None and str(source_value).strip():
            if 'option' in config:
                # Option-based mapping (e.g., checkbox to multi-select option)
                expected_option = config['option']
                target_has_option = False
                
                if isinstance(target_value, list):
                    target_has_option = expected_option in target_value
                elif isinstance(target_value, str):
                    target_has_option = expected_option == target_value
                
                action_type = "CLEANUP_ONLY" if target_has_option else "MIGRATE_AND_CLEANUP"
                
                analysis['migration_actions'].append({
                    'source_field': source_field,
                    'target_field': target_field,
                    'source_value': source_value,
                    'target_value': target_value,
                    'expected_option': expected_option,
                    'target_has_option': target_has_option,
                    'action': action_type
                })
                
                # Track cleanup needs
                analysis['cleanup_actions'].append({
                    'field': source_field,
                    'current_value': source_value,
                    'action': 'CLEAR_SOURCE_FIELD'
                })
                
            else:
                # Direct field mapping
                if target_value and str(target_value).strip():
                    if str(target_value).strip() != str(source_value).strip():
                        # Value conflict
                        analysis['conflicts'].append({
                            'source_field': source_field,
                            'target_field': target_field,
                            'source_value': source_value,
                            'target_value': target_value,
                            'issue': 'VALUE_CONFLICT'
                        })
                        analysis['migration_ready'] = False
                        action_type = "CONFLICT"
                    else:
                        action_type = "CLEANUP_ONLY"
                else:
                    action_type = "MIGRATE_AND_CLEANUP"
                
                analysis['migration_actions'].append({
                    'source_field': source_field,
                    'target_field': target_field,
                    'source_value': source_value,
                    'target_value': target_value,
                    'action': action_type
                })
    
    return analysis

def generate_migration_report_csv(analyses, filename):
    """Generate detailed CSV report for migration planning."""
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = [
            'Account_ID', 'Source_Field', 'Source_Value', 'Target_Field', 
            'Current_Target_Value', 'Expected_Option', 'Action_Required',
            'Migration_Ready', 'Has_Conflicts', 'Notes'
        ]
        
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        
        for analysis in analyses:
            account_id = analysis['account_id']
            
            if analysis['migration_actions']:
                for action in analysis['migration_actions']:
                    notes = []
                    if action['action'] == 'CLEANUP_ONLY':
                        notes.append("Target already correct - only source cleanup needed")
                    elif action['action'] == 'CONFLICT':
                        notes.append("CONFLICT: Different values in source and target")
                    
                    writer.writerow({
                        'Account_ID': account_id,
                        'Source_Field': action['source_field'],
                        'Source_Value': action['source_value'],
                        'Target_Field': action['target_field'],
                        'Current_Target_Value': action['target_value'],
                        'Expected_Option': action.get('expected_option', ''),
                        'Action_Required': action['action'],
                        'Migration_Ready': analysis['migration_ready'],
                        'Has_Conflicts': len(analysis['conflicts']) > 0,
                        'Notes': '; '.join(notes)
                    })
            else:
                # Account with no migration actions needed
                writer.writerow({
                    'Account_ID': account_id,
                    'Source_Field': 'NO_SOURCE_DATA',
                    'Source_Value': '',
                    'Target_Field': '',
                    'Current_Target_Value': '',
                    'Expected_Option': '',
                    'Action_Required': 'NO_ACTION',
                    'Migration_Ready': True,
                    'Has_Conflicts': False,
                    'Notes': 'No V-field data found'
                })

# Generate comprehensive migration report for specific accounts
print("📊 COMPREHENSIVE MIGRATION ANALYSIS REPORT")
print("="*60)

# Test accounts (add more account IDs as needed)
test_accounts = [6488]  # Replace with actual account IDs you want to analyze

print(f"🔍 Analyzing {len(test_accounts)} accounts for migration planning...")

# Analyze each account
analyses = []
for account_id in test_accounts:
    print(f"  📋 Analyzing account {account_id}...")
    analysis = analyze_account_migration_status(client, account_id, mapping)
    analyses.append(analysis)
    
    # Show summary for this account
    print(f"    • Migration ready: {analysis['migration_ready']}")
    print(f"    • Actions required: {len(analysis['migration_actions'])}")
    print(f"    • Conflicts: {len(analysis['conflicts'])}")
    print(f"    • Cleanup actions: {len(analysis['cleanup_actions'])}")

# Generate CSV report
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
report_filename = f"v_migration_report_{timestamp}.csv"
generate_migration_report_csv(analyses, report_filename)

print(f"\n📄 Detailed CSV report generated: {report_filename}")

# Display summary statistics
total_actions = sum(len(analysis['migration_actions']) for analysis in analyses)
total_conflicts = sum(len(analysis['conflicts']) for analysis in analyses)
total_cleanup = sum(len(analysis['cleanup_actions']) for analysis in analyses)
ready_accounts = sum(1 for analysis in analyses if analysis['migration_ready'])

print(f"\n📈 MIGRATION SUMMARY:")
print(f"  • Total accounts analyzed: {len(analyses)}")
print(f"  • Accounts ready for migration: {ready_accounts}")
print(f"  • Accounts with conflicts: {len(analyses) - ready_accounts}")
print(f"  • Total migration actions: {total_actions}")
print(f"  • Total cleanup actions: {total_cleanup}")
print(f"  • Total conflicts: {total_conflicts}")

# Show detailed breakdown for first account (as example)
if analyses:
    example_analysis = analyses[0]
    print(f"\n🔍 DETAILED BREAKDOWN FOR ACCOUNT {example_analysis['account_id']}:")
    
    print(f"\n  📥 SOURCE FIELDS WITH DATA:")
    for field, value in example_analysis['source_fields'].items():
        if value is not None and str(value).strip():
            print(f"    • {field}: {value}")
    
    print(f"\n  📤 CURRENT TARGET FIELD VALUES:")
    for field, value in example_analysis['target_fields'].items():
        print(f"    • {field}: {value}")
    
    print(f"\n  ⚡ REQUIRED ACTIONS:")
    for action in example_analysis['migration_actions'][:10]:  # Show first 10
        if 'expected_option' in action:
            print(f"    • {action['action']}: {action['source_field']} -> {action['target_field']} (option: {action['expected_option']})")
        else:
            print(f"    • {action['action']}: {action['source_field']} -> {action['target_field']}")
    
    if example_analysis['conflicts']:
        print(f"\n  ⚠️  CONFLICTS:")
        for conflict in example_analysis['conflicts']:
            print(f"    • {conflict['source_field']} vs {conflict['target_field']}: '{conflict['source_value']}' != '{conflict['target_value']}'")

print(f"\n✅ Analysis complete! Use the CSV report for detailed migration planning.")
print(f"💡 The report shows exactly what will happen to each field during migration.")